In [10]:
from glob import glob
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from data_processing import load_dataframe, load_clean_dataframe

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
country_groups = load_dataframe("data/country_groups.csv", None)

In [21]:
column_rename = {'country': 'area', 'countrycode': 'areacode'}
duplicate_cols = ["yearcode", "elementgroup"]

csv_files = glob('data/production/Production_Livestock_E_All_Data_(Normalized).csv')
len(csv_files)

1

In [22]:
from mappings import is_unique_mapping, get_mapping
def load_and_update_csv(file, item_groups, country_groups, col_rename, duplicate_cols, drop_elements, drop=False,drop_agg=False):
    df = load_clean_dataframe(file, col_rename, duplicate_cols)
    #df = df[~df.areacode.isin(country_groups.countrygroupcode.unique())]
    
    if drop_elements is not None:
        df = df[~df.elementcode.isin(drop_elements)]  # Drop elements
    
    df = df[~df['value'].isna()]  # Drop all NaN values  
    
    item_unique = is_unique_mapping(df, 'itemcode', ['item'])
    element_unique = is_unique_mapping(df, 'elementcode',  ['element', 'unit'])
    
    print(item_unique, element_unique)
    item_mapping = get_mapping(df, ['itemcode', 'item'])
    element_mapping = get_mapping(df, ['elementcode', 'element', 'unit'])
    
    if item_groups is not None:
        item_groups = load_dataframe(item_groups)
        
        if not drop_agg:
            df = df[~df.itemcode.isin(item_groups['itemgroupcode'].unique())]  # Drop all itemgroups present in DF
            item_mapping = get_mapping(item_groups, ['itemgroupcode', 'itemgroup'])
            groups = item_groups.groupby(['itemgroupcode'])['itemcode'].apply(set)
            dfs = []
            for code, group in groups.iteritems():

                new_df = df[df.itemcode.isin(group)].groupby(['areacode', 'elementcode', 'year'])['value'].sum().reset_index()
                new_df = new_df.assign(itemcode=code)
                new_df = new_df.assign(item=item_mapping[code])
                new_df = new_df.assign(element=new_df.elementcode.apply(lambda x: element_mapping[x][0]))
                new_df = new_df.assign(unit=new_df.elementcode.apply(lambda x: element_mapping[x][1]))
                dfs.append(new_df)

            new_df = pd.concat(dfs).reset_index(drop=True)
            if drop:
                df = new_df
            else:
                df = pd.concat([df.drop('flag', axis=1), new_df]).reset_index(drop=True)
            
        else:
            df = df[~df.itemcode.isin(item_groups.itemgroupcode.unique())]
    return df, item_mapping, element_mapping

In [27]:
csv = "data/production/Production_Livestock_E_All_Data_(Normalized).csv"
item_groups = "data/item_groups/Production_Livestock_All_data.csv"
df, item_mapping, element_mapping = load_and_update_csv(csv, item_groups, country_groups, column_rename, duplicate_cols, None, drop=True, drop_agg=False)

True True


In [20]:
df.to_csv("data_cleaned/environment/Environment_Emissions_by_Sector.csv")

In [28]:
df.item.unique()

array(['Cattle and Buffaloes', 'Sheep and Goats', 'Live Animals',
       'Poultry Birds'], dtype=object)

In [29]:
df

,areacode,elementcode,year,value,itemcode,item,element,unit
0,1,5111,1992,566777.0,1746,Cattle and Buffaloes,Stocks,Head
1,1,5111,1993,499199.0,1746,Cattle and Buffaloes,Stocks,Head
2,1,5111,1994,502214.0,1746,Cattle and Buffaloes,Stocks,Head
3,1,5111,1995,504391.0,1746,Cattle and Buffaloes,Stocks,Head
4,1,5111,1996,508114.0,1746,Cattle and Buffaloes,Stocks,Head
...,...,...,...,...,...,...,...,...
73746,5817,5112,2013,2955562.0,2029,Poultry Birds,Stocks,1000 Head
73747,5817,5112,2014,3094149.0,2029,Poultry Birds,Stocks,1000 Head
73748,5817,5112,2015,3253248.0,2029,Poultry Birds,Stocks,1000 Head
73749,5817,5112,2016,3357832.0,2029,Poultry Birds,Stocks,1000 Head


In [26]:
df[['elementcode', 'element', 'unit']].drop_duplicates()

,elementcode,element,unit
0,5111,Stocks,Head
171,5112,Stocks,1000 Head
684,5114,Stocks,No


In [477]:
df

,areacode,area,itemcode,item,elementcode,element,year,unit,value,flag
0,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1990,%,97.22,Fc
1,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1991,%,97.57,Fc
2,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1992,%,97.84,Fc
3,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1993,%,98.41,Fc
4,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1994,%,98.86,Fc
...,...,...,...,...,...,...,...,...,...,...
722,229,United Kingdom,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,2005,%,88.29,Fc
723,229,United Kingdom,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,2006,%,88.68,Fc
724,229,United Kingdom,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,2007,%,88.64,Fc
725,229,United Kingdom,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,2008,%,88.49,Fc


In [457]:
t = load_clean_dataframe(csv, column_rename, duplicate_cols)

In [469]:
t2 = load_clean_dataframe("data/environment/Environment_AirClimateChange_E_All_Data.csv", column_rename, duplicate_cols)

In [470]:
t2.item.unique()

array(['Ammonia (NH3) emissions from agriculture'], dtype=object)

In [450]:
t.year.min(), t2.year.max()t.ele

(1961, 2002)

In [472]:
t2

,areacode,area,itemcode,item,elementcode,element,year,unit,value,flag
0,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1990,%,97.22,Fc
1,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1991,%,97.57,Fc
2,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1992,%,97.84,Fc
3,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1993,%,98.41,Fc
4,3,Albania,6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,1994,%,98.86,Fc
...,...,...,...,...,...,...,...,...,...,...
802,5000,World + (Total),6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,2005,%,63.39,Fc
803,5000,World + (Total),6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,2006,%,64.11,Fc
804,5000,World + (Total),6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,2007,%,80.71,Fc
805,5000,World + (Total),6730,Ammonia (NH3) emissions from agriculture,7203,% of total NH3 emissions from agriculture,2008,%,84.08,Fc


In [471]:
t2[['elementcode', 'element', 'unit']].drop_duplicates()

,elementcode,element,unit
0,7203,% of total NH3 emissions from agriculture,%


In [459]:
t.item.unique()

array(['Forest land', 'Burning Biomass', 'Land Use total', 'Cropland',
       'Grassland'], dtype=object)

In [387]:
t[(t.item == "Rice Cultivation") & (t.areacode == 181) & (t.year == 1961)]

,areacode,area,itemcode,item,elementcode,element,year,unit,value,flag


In [388]:
df[(df.areacode == 181) & (df.year == 1961)]

,areacode,area,itemcode,item,elementcode,element,year,unit,value,flag
